In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!git clone -b main_dev https://github.com/UM-Titan/DSPS.git
%cd DSPS

Cloning into 'DSPS'...
remote: Enumerating objects: 809, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 809 (delta 32), reused 25 (delta 12), pack-reused 752
Receiving objects: 100% (809/809), 378.77 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (184/184), done.
Checking out files: 100% (150/150), done.
/content/drive/My Drive/DSPS


In [ ]:
## Uncomment the lines below to download training data
!gdown 'https://drive.google.com/uc?id=1I1onAA61iui4LjOxejkxioaBjU1Ic_6X'
!unzip 'training_data.zip' -d './cvat'

**Downloading the second batch of training data**

In [ ]:
!gdown 'https://drive.google.com/uc?id=11ax5I_3NeX1bEIFRAvzBMe2tRvBLn7yT'
!unzip 'td4.zip' -d './cvat'

In [ ]:
# cvat folder should contain 5 folders: td1, td2, td3, td4_1, td4_2_3, td4_4
!ls cvat

td1  td2  td3  td4_1  td4_2_3  td4_4


In [ ]:
!pip install -U PyYAML

     |████████████████████████████████| 596 kB 8.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install -qr dspYolov5/requirements.txt  # install dependencies for yolov5

**In order to use Albumentation packege which is integrated with Yolov5, it should be istalled first.**

In [ ]:
!pip install -U albumentations
!pip install "opencv-python-headless<4.3"

**In order to virtualize and track the training performance, wandb package should be install.**

In [ ]:
# !pip install wandb

In [ ]:
import os
from dsps.simple_fxn import cvat2yolo
# location of cvat annotations
annotation_folder = 'cvat/'
output_path = 'data/training'
val_path = 'data/validation'
for annts in os.listdir(annotation_folder):
  json_file = os.path.join(annotation_folder,annts,'annotations','instances_default.json')
  img_path = os.path.join(annotation_folder,annts,'images')
  if(os.path.isfile(json_file)):
    cvat2yolo(annts,json_file,img_path, output_path, val_path)

{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['ALLIGATOR', 'SEALING', 'TRANSVERSE', 'PATCHING', 'LONGITUDINAL', 'BLOCK', 'MANHOLE']
{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['TRANSVERSE', 'PATCHING', 'LONGITUDINAL', 'ALLIGATOR', 'SEALING']
{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['LONGITUDINAL', 'TRANSVERSE', 'PATCHING', 'ALLIGATOR', 'BLOCK', 'SEALING', 'MANHOLE']
{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['BLOCK', 'LONGITUDINAL', 'TRANSVERSE', 'ALLIGATOR']
{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['ALLIGATOR', 'LONGITUDINAL', '

**Confirmimg number of images and annotations in each folder**

In [ ]:
import os
print (len(os.listdir('/content/drive/MyDrive/DSPS/data/training')))
print (len(os.listdir('/content/drive/MyDrive/DSPS/data/validation')))

638
154


In [ ]:
import json
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla T4)


**Modifications**


---


- Adding 2 types of data augmentation which is mentioned in the training output cell (Albumentation).

1.   Invert Image (probibility=0.5)
2.   Random Sized Bounding Box Safe Crop (probibility=0.5, height=640, width=640, erosion rate=0.2)

The Following path represent the file for midification:

*DSPS/dspYolov5/utils/augmentations.py*

- In order to prevent overfitting, the object loss gain "obj" decreased to 0.1, this is also mentioned in the training output cell (hyperparameters).

The Following path represent the file for midification:

*DSPS/dspYolov5/data/hyps/hyp.scratch.yaml*


---

**Training**

In [ ]:
!python -i dspYolov5/train_dsps.py --data dspYolov5/data/dsps.yaml --cfg dspYolov5/models/dsps_model.yaml --batch-size 8 --device 0


train: weights=yolov5s.pt, cfg=dspYolov5/models/dsps_model.yaml, data=dspYolov5/data/dsps.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=150, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=weights/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
fatal: ambiguous argument 'main_dev..origin/master': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'
github: Command 'git rev-list main_dev..origin/master --count' returned non-zero exit status 128.
YOLOv5 🚀 119ab33 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.0

**Before downloading the test data, delete the existing one**

In [ ]:
## Uncomment the lines below to download testing data
!gdown 'https://drive.google.com/uc?id=1TtDGMgTyrH6bclBf76ggZk28N9aalOzc'
!unzip 'dsps_test.zip' -d './data/test_images'

**Modifications**


---


- in order to increase the accuracy in detection the following changes had been made:

1.   The IOU threshold set to 0.999
2.   The Confidence threshold set to 0.35
3.   Test Time Augmentation applied


---

**Detection**

In [ ]:
!python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp/weights/best.pt' --img 640 --conf 0.35 --iou-thres 0.999 --augment --source data/test_images/images

**The compelete folder of the trained model could be accessed by the following link:**

https://drive.google.com/drive/folders/1jFuHKu05hL0kiaasevJAr_PRDZyUKGdC?usp=sharing